
# How to build an Automated Quant Database

Quants analyze market data, often using internal databases at firms like Jane Street, Man Group, and Goldman Sachs. Creating your own stock price database is essential for integrating various data points—like stock prices, economic trends, and custom analytics—to enhance market research efficiency.

With the rise of free market data, now is an ideal time to start storing and analyzing it. This issue will guide you on how to:

- Use SQLite to build a database
- Download stock data for free
- Store the data in a database
- Automate the entire process

Sources:
- https://www.sqlite.org/
- https://docs.python.org/3/library/sqlite3.html
- https://www.pyquantnews.com/the-pyquant-newsletter/how-to-build-an-automated-quant-database

In [28]:
import pandas as pd
import yfinance as yf
import sqlite3

In [23]:
def get_stock_data(symbol, start, end):
    data = yf.download(symbol, start=start, end=end)
    data.reset_index(inplace=True)
    data.rename(columns={
        'Date': 'date',
        'Open': 'open',
        'High': 'high',
        'Low': 'low',
        'Close': 'close',
        'Adj Close': 'adj_close',
        'Volume': 'volume'
    }, inplace=True)
    data['symbol'] = symbol
    return data

In [29]:
def save_data_range(symbol, start, end, con):
    try:
        data = get_stock_data(symbol, start, end)
        if isinstance(data,pd.DataFrame) and not data.empty:
            data.to_sql('stock_data', con, if_exists='append', index=False)
            print(f"Data for {symbol} from {start} to {end} saved successfully.")
        else:
            print(f"No data found for {symbol} from {start} to {end}.")
    except Exception as e:
        print(f"Error saving data for {symbol}: {e}")

In [32]:
def retrieve_stock_data(symbol, start_date, end_date, con):
    """Retrieve stock data from the database within a date range."""
    query = f"""
    SELECT * FROM stock_data
    WHERE symbol = '{symbol}' AND date BETWEEN '{start_date}' AND '{end_date}'
    """
    return pd.read_sql_query(query, con)

In [33]:
# Main execution block
if __name__ == '__main__':
    # Connect to the SQLite database
    con = sqlite3.connect("08_market_data.sqlite")

    con.execute("DROP TABLE IF EXISTS stock_data")

    # Check if the stock_data table exists and create it if not
    con.execute('''
    CREATE TABLE IF NOT EXISTS stock_data (
        date TEXT,
        open REAL,
        high REAL,
        low REAL,
        close REAL,
        adj_close REAL,
        volume INTEGER,
        symbol TEXT
    )
    ''')
    
    # User input for gathering stock data
    gather_data = input("Do you want to gather stock data? (yes/no): ").strip().lower()
    if gather_data == 'yes':
        symbol = input("Enter stock symbol: ").strip().upper()
        start_date = input("Enter start date (YYYY-MM-DD): ").strip()
        end_date = input("Enter end date (YYYY-MM-DD): ").strip()

        # Save the stock data for the specified range
        save_data_range(symbol, start_date, end_date, con)
        print(f"Data for {symbol} from {start_date} to {end_date} saved to database.")
    
    # User input for retrieving stock data
    retrieve_data = input("Do you want to retrieve stock data? (yes/no): ").strip().lower()
    if retrieve_data == 'yes':
        symbol = input("Enter stock symbol: ").strip().upper()
        start_date = input("Enter start date (YYYY-MM-DD): ").strip()
        end_date = input("Enter end date (YYYY-MM-DD): ").strip()

        # Retrieve the stock data for the specified range
        df_stock = retrieve_stock_data(symbol, start_date, end_date, con)

        # Display the retrieved data
        if not df_stock.empty:
            print(f"Retrieved data for {symbol} from {start_date} to {end_date}:")
            print(df_stock)
        else:
            print(f"No data found for {symbol} from {start_date} to {end_date}.")
    
    # Close the database connection
    con.close()

[*********************100%***********************]  1 of 1 completed


Error saving data for IBM: table stock_data has no column named ('date', '')
Data for IBM from 2020-01-01 to 2025-04-03 saved to database.
No data found for IBM from 2020-01-01 to 2025-04-03.
